In [51]:
import cv2
import numpy as np
import math
from PIL import Image

pic = cv2.imread('xray.tif', 0)

pic = np.array(pic)

sigma = 1

T=0.3

sHalf = round(math.sqrt(-math.log(T) * 2 * sigma**2)) 

N= 2*sHalf +1 

[X, Y]=np.mgrid[-sHalf:sHalf+1, -sHalf:sHalf+1]


fx = (1/(sigma*np.sqrt(2*np.pi)))*(-X/(sigma**2))*(np.exp(-(X**2 + Y**2)/(2*(sigma**2))))

fy = (1/(sigma*np.sqrt(2*np.pi)))*(-Y/(sigma**2))*(np.exp(-(X**2 + Y**2)/(2*(sigma**2))))


fx = np.round(fx*255)

fy = np.round(fy*255)


temp1 = np.zeros(pic.shape)

temp2 = np.zeros(pic.shape)


def convolve2d(image, kernel): #function for convolution
    
    kernel = np.flipud(np.fliplr(kernel))  #flips the filter/kernel
    image_h = image.shape[0]
    image_w = image.shape[1]
    
    kernel_h = kernel.shape[0]
    kernel_w = kernel.shape[1]
    
    h = kernel_h//2
    w = kernel_w//2
    
    image_conv = np.zeros(image.shape)
    
    for i in range(h, image_h - h):
        for j in range(w, image_w - w):
            sum = 0
            
            for m in range(kernel_h):
                for n in range(kernel_w):
                    sum = sum + kernel[m][n] * image[i-h+m][j-w+n] #sum of multiplication of the filter to the image
                    
            image_conv[i][j] = sum
    
    
    return image_conv


fx = convolve2d(pic, fx)

fy = convolve2d(pic, fy)

fx = np.round(fx)

fy = np.round(fy)

cv2.imwrite('xray _fx_2.jpg', fx)

cv2.imwrite('xray _fy_2.jpg', fy)



#Gradient Magnitude

M = np.zeros(pic.shape)

M = np.sqrt((fx**2) + (fy**2))/255

cv2.imwrite('xray _magnitude_2.jpg', M)



#Gradient Direction

D = np.zeros(pic.shape)

D = np.round(np.degrees(np.arctan2(fy, fx))) + 180



#Non-Maxima Suppression

C = np.array(Image.open("xray.tif").convert("RGB"))

C = np.zeros(C.shape)


for y in range(pic.shape[0]):
    for x in range(pic.shape[1]):
        
        check = D[y,x].item()

        if check in np.arange(0.0, 22):
            C[y,x] = [0, 0, 255]
        if check in np.arange(157, 202):
            C[y,x] = [0, 0, 255] 
            if check == 180.0:
                C[y,x] = [0, 0, 0] 
        if check in np.arange(337, 360):
            C[y,x] = [0, 0, 255]
    
        if check in np.arange(22, 67):
            C[y,x] = [0, 255, 0]
        if check in np.arange(202, 247):
            C[y,x] = [0, 255, 0]

        if check in np.arange(67, 112):
            C[y,x] = [255, 0, 0]
        if check in np.arange(247, 292):
            C[y,x] = [255, 0, 0]

        if check in np.arange(112, 157):
            C[y,x] = [0, 255, 255]
        if check in np.arange(292, 337):
            C[y,x] = [0, 255, 255]

            
cv2.imwrite('xray _quantized_2.jpg', C)
            
Mcop = M.copy()
            
for y in range(pic.shape[0] - 1):
        for x in range(pic.shape[1] - 1):
            
            check = D[y,x].item()
            
            if y == 0 or x == 0:
                Mcop[y, x] = 0
                continue

            if check in np.arange(0, 22) or check in np.arange(158, 202) or check in np.arange(338, 360): #North to South
                if M[y, x] <= M[y-1, x] or M[y, x] <= M[y+1, x]:
                    Mcop[y, x] = 0
            
            if check in np.arange(23, 67) or check in np.arange(203, 247): #North-West to South-East
                if M[y, x] <= M[y-1, x-1] or M[y, x] <= M[y+1, x+1]:
                    Mcop[y, x] = 0           
            
            if check in np.arange(68, 112) or check in np.arange(248, 292): #East to West
                if M[y, x] <= M[y, x-1] or M[y, x] <= M[y, x+1]:
                    Mcop[y, x] = 0 
                    
            if check in np.arange(113, 157) or check in np.arange(293, 337): #North-East to South-West
                if M[y, x] <= M[y-1, x+1] or M[y, x] <= M[y+1, x-1]:
                    Mcop[y, x] = 0

            
#cv2.imwrite('xray _NonMaxima_2.jpg', Mcop)


#Hysteresis Thresholding

visit = np.zeros(M.shape)

def hysteresis(img, Tl, Th):
    #Th = Th*255
    #Tl = Th*255    
    Thll = 255
    
    for y in range(pic.shape[0] - 1):
        for x in range(pic.shape[1] - 1):
            #check = img[y,x].item()
            if (img[y,x] >= Th):
                #print(img[y,x], ", ", Th)
                img[y, x] = Thll
                if ((img[y+1, x-1] >= Tl) and visit[y+1, x-1] != 255):   
                    img[y+1, x-1] = Thll
                    visit[y+1, x-1] = 255
                elif ((img[y+1, x] >= Tl) and visit[y+1, x] != 255):
                    img[y+1, x] = Thll
                    visit[y+1, x] = 255
                elif ((img[y+1, x+1] >= Tl) and visit[y+1, x+1] != 255):
                    img[y+1, x+1] = Thll
                    visit[y+1, x+1] = 255
                elif ((img[y, x-1] >= Tl) and visit[y, x-1] != 255):
                    img[y, x-1] = Thll
                    visit[y, x-1] = 255
                elif ((img[y, x+1] >= Tl) and visit[y, x+1] != 255):
                    img[y, x+1] = Thll
                    visit[y, x+1] = 255
                elif ((img[y-1, x-1] >= Tl) and visit[y-1, x-1] != 255):
                    img[y-1, x-1] = Thll
                    visit[y-1, x-1] = 255
                elif ((img[y-1, x] >= Tl) and visit[y-1, x] != 255):
                    img[y-1, x] = Thll
                    visit[y-1, x] = 255
                elif ((img[y-1, x+1] >= Tl) and visit[y-1, x+1] != 255):
                    img[y-1, x+1] = Thll
                    visit[y-1, x+1] = 255

    return img


Mcop2 = np.zeros(M.shape)

Mcop2 = hysteresis(Mcop, 5, 20)

cv2.imwrite('xray _Final_1_5, 20.jpg', Mcop2)

print('Done')

85.83698551036791 ,  20
47.166011912589795 ,  20
30.103751389998973 ,  20
26.87906903470136 ,  20
21.890934082444318 ,  20
35.72771134482664 ,  20
23.51470945893444 ,  20
110.63893268468173 ,  20
80.01401126822142 ,  20
33.97624812086565 ,  20
255.0 ,  20
255.0 ,  20
43.32681381175883 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
35.66817575955644 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
42.597251961406975 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
35.582496161233635 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
35.57707662796049 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
48.67489225962361 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  2

255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
28.310856679387143 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
49.612996089808014 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.138010944816106 ,  20
21.165055020426607 ,  20
51.302779295070295 ,  20
51.770948078772996 ,  20
255.0 ,  20
20.237618045608492 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
43.622287423006355 ,  20
255.0 ,  20
255.0 ,  20
26.736051888568653 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.37683776028077 ,  20
23.600748396503192 ,  20
255.0 ,  20
255.0 ,  20
21.99304353335513 ,  20
255.0 ,  20
26.117515161887997 ,  20
255.0 ,  20
22.785094313515955 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
60.641662591242216 ,  20
46.96684819010061 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255

33.586427844183795 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
22.678117061697435 ,  20
23.261867818926774 ,  20
23.49966027550624 ,  20
23.434628045236995 ,  20
22.53573692332762 ,  20
20.75093509193122 ,  20
20.371579738779243 ,  20
23.22468194821062 ,  20
28.139276008575855 ,  20
255.0 ,  20
42.65632533989851 ,  20
255.0 ,  20
255.0 ,  20
35.947364014948604 ,  20
47.442463990243915 ,  20
255.0 ,  20
52.976328925044164 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
28.035403827003734 ,  20
29.350694505818023 ,  20
28.35463721773437 ,  20
27.229060679401094 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
86.3986011321106 ,  20
255.0 ,  20
31.647355248022553 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
42.1498976462322 ,  20
255.0 ,  20
43.9571818152745 ,  20
255.0 ,  20
50.61027183816036 ,  20
255.0 ,  20
54.395

255.0 ,  20
27.16475145561838 ,  20
28.465522425133777 ,  20
27.986436270951327 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
27.80734125379065 ,  20
255.0 ,  20
28.01531327288306 ,  20
255.0 ,  20
31.99317497382536 ,  20
255.0 ,  20
27.270880631387396 ,  20
255.0 ,  20
20.467397392363218 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
23.104160445936408 ,  20
23.389766654670552 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
24.69540269490953 ,  20
26.97117261538947 ,  20
255.0 ,  20
22.916878367481853 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
28.525427927495695 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
21.161596083209965 ,  20
255.0 ,  20
20.51018970131691 ,  20
23.09980784866265 ,  20
255.0 ,  20
255.0 ,  20
34.02469035888035 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,

255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
21.41526725643984 ,  20
22.86943869996299 ,  20
23.918931709372956 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
29.343650277545887 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
39.14054860595253 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
26.062673405104583 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
21.496636003736125 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.259733683334694 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
46.015496445331785 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
27.81139449893504 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
22.25266603607728 ,  20
255.0 ,  20
21.6853529

...

255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
36.98520522605256 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
28.021732654516008 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
29.858824559505948 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
70.07493116992566 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
23.58979604635552 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.441161329754117 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.168778426613688 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20


255.0 ,  20
255.0 ,  20
21.2366646308776 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
22.484882357720327 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
22.715147285150547 ,  20
255.0 ,  20
21.28385337423603 ,  20
30.46829425841396 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.659279477452486 ,  20
22.734723800437934 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.67638458874444 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
20.872289207161845 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
25.70650004492608 ,  20


255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
26.73785221840685 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
37.693848578837795 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
65.70378063472657 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
22.268465418509376 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
21.642968141479283 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
74.1768072846

255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
31.436809677371407 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
21.101351879303355 ,  20
20.45053859993172 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
27.354134206886926 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
2

255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
26.43040813973387 ,  20
255.0 ,  20
255.0 ,  20
27.772765253240642 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
23.852714794667296 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
255.0 ,  20
25